# Groupby Extras

* deeper dive into the `groupby` object
* using multiple aggregators 
* custom aggregators

In [1]:
import pandas as pd
import numpy as np

stock = pd.DataFrame({
    'item_no': pd.Series([1, 2, 2, 4, 5, 6, 7, 8, 9, 10], dtype='Int64'),
    'cost_class': pd.Series(['1st', '2nd', '3rd', '4th', '4th', '3rd', '2nd', np.nan, '1st', '3rd'], dtype='string'),
    'cost': pd.Series([10.99, np.nan, 2.99, np.nan, 2.99, 2.45, 5.99, 5.99, 3.00, None], dtype='float64'),
    'stock_code': pd.Series(['a', 'a', 'c', 'b', 'a', 'b', np.nan, np.nan, 'a', 'c'], dtype='string'),
    'priority_code': pd.Series([np.nan, None, 'a', 'b', None, 'a', 'e', None, 'a', 'd'], dtype='string'),
    'tax_rate': pd.Series([0, 0, 20, 20, 20, 0, 20, 20, 5, 20])
})

stock

item_no cost_class   cost stock_code priority_code  tax_rate
0        1        1st  10.99          a          <NA>         0
1        2        2nd    NaN          a          <NA>         0
2        2        3rd   2.99          c             a        20
3        4        4th    NaN          b             b        20
4        5        4th   2.99          a          <NA>        20
5        6        3rd   2.45          b             a         0
6        7        2nd   5.99       <NA>             e        20
7        8       <NA>   5.99       <NA>          <NA>        20
8        9        1st   3.00          a             a         5
9       10        3rd    NaN          c             d        20

In [2]:
grouped_stock = (
    stock
    .groupby("stock_code")
)

In [4]:
grouped_stock.cost_class.min()

stock_code
a    1st
b    3rd
c    3rd
Name: cost_class, dtype: string

In [6]:
grouped_stock

In [9]:
pd.DataFrame(grouped_stock).loc[0, 1]

item_no cost_class   cost stock_code priority_code  tax_rate
0        1        1st  10.99          a          <NA>         0
1        2        2nd    NaN          a          <NA>         0
4        5        4th   2.99          a          <NA>        20
8        9        1st   3.00          a             a         5

* splitting
* split-apply-combine
* groupby = split

In [10]:
grouped_stock.count()

item_no  cost_class  cost  priority_code  tax_rate
stock_code                                                    
a                 4           4     3              1         4
b                 2           2     1              2         2
c                 2           2     1              2         2

## Applying Multiple Aggregators

using the `.agg()` method and a list of strings

so now

* we apply each aggregate function to our group
* combine each group summary
* output

In [13]:
multiple_aggs = (
    grouped_stock
    .agg(["count", "min", "max"])
)

In [15]:
# columns are now a MultiIndex

multiple_aggs.columns

MultiIndex([(      'item_no', 'count'),
            (      'item_no',   'min'),
            (      'item_no',   'max'),
            (   'cost_class', 'count'),
            (   'cost_class',   'min'),
            (   'cost_class',   'max'),
            (         'cost', 'count'),
            (         'cost',   'min'),
            (         'cost',   'max'),
            ('priority_code', 'count'),
            ('priority_code',   'min'),
            ('priority_code',   'max'),
            (     'tax_rate', 'count'),
            (     'tax_rate',   'min'),
            (     'tax_rate',   'max')],
           )

### Column formula

```python
(original_column_name, aggregate_function_name)
```

In [16]:
# what is the max cost for each stock code??

multiple_aggs.loc[:, ("cost", "max")]

stock_code
a    10.99
b     2.45
c     2.99
Name: (cost, max), dtype: float64

Task:

Get the `mean`, `count` and `max` for all variables in `stock`

* what has gone wrong??
* why??

In [19]:
more_aggs = (
    grouped_stock
    .agg(["mean", "count", "max"])
)

more_aggs

/var/folders/xp/pxq000_97vz2g8qfqw7qtgvm0000gn/T/ipykernel_17093/3412093515.py:2: FutureWarning: ['cost_class', 'priority_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_stock


item_no            cost              tax_rate          
              mean count max  mean count    max     mean count max
stock_code                                                        
a             4.25     4   9  5.66     3  10.99     6.25     4  20
b              5.0     2   6  2.45     1   2.45    10.00     2  20
c              6.0     2  10  2.99     1   2.99    20.00     2  20

In [21]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_no        10 non-null     Int64  
 1   cost_class     9 non-null      string 
 2   cost           7 non-null      float64
 3   stock_code     8 non-null      string 
 4   priority_code  6 non-null      string 
 5   tax_rate       10 non-null     int64  
dtypes: Int64(1), float64(1), int64(1), string(3)
memory usage: 618.0 bytes


* `cost_class`
    * string
* `priority_code`
    * string

Define a dictionary of aggregations defining which aggregations to perform on each column

```python
grouped_df.agg({
    "column_nmae": [aggregate_operations]
```

In [23]:
grouped_stock.agg({
    "cost_class": ["count", "min", "max"],
    "cost": ["mean", "median"],
    "stock_code": ["count"]
})

cost_class            cost        stock_code
                count  min  max  mean median      count
stock_code                                             
a                   4  1st  4th  5.66   3.00          4
b                   2  3rd  4th  2.45   2.45          2
c                   2  3rd  3rd  2.99   2.99          2

In [24]:
multiple_group = (
    stock
    .groupby(["cost_class", "stock_code"])
)

In [27]:
mean_multi_group = (
    multiple_group
    .agg("mean")
)

mean_multi_group

item_no   cost  tax_rate
cost_class stock_code                          
1st        a               5.0  6.995       2.5
2nd        a               2.0    NaN       0.0
3rd        b               6.0  2.450       0.0
           c               6.0  2.990      20.0
4th        a               5.0  2.990      20.0
           b               4.0    NaN      20.0

In [31]:
mean_multi_group.loc[("3rd", "c"), "cost"]

2.99

In [29]:
multiple_group_no_drop = (
    stock
    .groupby(["cost_class", "stock_code"], dropna=False)
)

In [30]:
multiple_group_no_drop.agg("mean")

item_no   cost  tax_rate
cost_class stock_code                          
1st        a               5.0  6.995       2.5
2nd        a               2.0    NaN       0.0
           <NA>            7.0  5.990      20.0
3rd        b               6.0  2.450       0.0
           c               6.0  2.990      20.0
4th        a               5.0  2.990      20.0
           b               4.0    NaN      20.0
<NA>       <NA>            8.0  5.990      20.0

## Bonus - Custom Aggregation

Get the sum of each stock code group (minus a £2.00 restocking fee per item)

In [34]:
stock_costs = grouped_stock.agg({
    "item_no": ["count"],
    "cost": ["sum"]
})

In [35]:
stock_costs[("cost", "sum")] = stock_costs[("cost", "sum")] - 2*stock_costs[("item_no", "count")]

In [36]:
stock_costs

item_no  cost
             count   sum
stock_code              
a                4  8.98
b                2 -1.55
c                2 -1.01

In [37]:
def sum_minus_restocking(rows, restocking_fee=2.00):
    return(rows.sum() - (rows.count() * restocking_fee))

In [38]:
# different results due to handling of NA values

stock_costs = grouped_stock.agg({
    "item_no": ["count"],
    "cost": sum_minus_restocking
})

stock_costs

item_no                 cost
             count sum_minus_restocking
stock_code                             
a                4                10.98
b                2                 0.45
c                2                 0.99